# SARSIM PLUS DEMO
Demo Code to show how to use the SARSIM PLUS database.
SARSIM PLUS includes tomographic voxels generated from SARSIM.

SAR processing was carried out by Politecnico di Milano, DLR,
and University of Rennes. Please refer to the SARSIM manual for 
details about data processing.

The original script and related routines were developed by Mauro
Mariotti d'Alessandro and Stefano Tebaldini (PoliMi) using
Matlab 2015b on a 64 bit machine under Windows 10.

The demo code was converted to Python 3.6 by Guido Riembauer (ESTEC).
Required packages are numpy, matplotlib, os, math, gdal.

All Matlab and Python codes provided with SARSIM are intended for 
educational purposes only.

### Implemented Functions:
- data selection
- load TomoSAR voxels and ancillary data
- display loaded data

### How to use this demo

The Jupyter notebook is split into two parts: 1) Preparation and 2) Processing. In part 1), modules are imported and functions are defined. There is no user input required apart from running the sections. In the beginning of part 2), the user can then define the dataset to be analysed.
Run cells by pressing Ctrl+Enter or Shift+Enter to run a cell and proceed to the next one.


### Important: Set the path to the SARSIM database
Adapt the following string to your path to the SARSIM database and run the cell
by pressing Ctrl+Enter or Shift+Enter.

In [ ]:
sarsim_path = "D:\\SARSIM\\"

# 1) Preparation
Simply run the following sections to import packages/modules and to
define functions required for data loading

### Import packages

In [ ]:
import numpy as np
import gdal
import os
from math import ceil
%matplotlib notebook
import matplotlib.pyplot as plt

### Define functions to load SARSIM data

In [ ]:
def SARSIM_PLUS_DATA_SELECTION(parent_folder ,Operator, Site, Pol_pair, Band, Sensor, Z):
###############################################################################
# Operator = {DLR,ONERA,JPL}
# Site = {Lope, Remningstorp, Traunstein, Krycklan_SW, Krycklan_NE}
# Pol_pair = {HHHH,HVHV,VHVH,VVVV,HHVV}
# Band = {P,L}
# Sensor = {Airborne, Spaceborne}
# Z = Section height w.r.t DTM (the closest section will be loaded)
###############################################################################
  ID = {"dataset":{}}
    
  ID["dataset"]["Operator"] = Operator
  ID["dataset"]["Site"] = Site
  ID["dataset"]["Pol_pair"] = Pol_pair
  ID["dataset"]["Band"] = Band
  ID["dataset"]["Sensor"] = Sensor
  ID["Z"] = Z
    
  if isinstance(Z,str):
    ID["load_all_sections"] = True
  else:
    ID["load_all_sections"] = False
    
  ID["parent_folder"] = parent_folder
  
  if Operator == "DLR":
    ID["operator_ID"] = 0
  elif Operator == "ONERA":
    ID["operator_ID"] = 1
  elif Operator == "JPL":
    ID["operator_ID"] = 2
  else:
    raise ValueError("Operator " + Operator + " does not exist in SARSIM. Available choices:{DLR,ONERA,JPL}")
  
  if Site == "Lope":
    ID["site_ID"] = 0
  elif Site == "Remningstorp":
    ID["site_ID"] = 1
  elif Site == "Traunstein":
    ID["site_ID"] = 2
  elif Site == "Krycklan_SW":
    ID["site_ID"] = 3
  elif Site == "Krycklan_NE":
    ID["site_ID"] = 4
  else:
    raise ValueError("Site " + Site + " does not exist in SARSIM. Available choices:{Lope,Remningstorp,Traunstein,Krycklan_SW,Krycklan_NE}")
  
  if Pol_pair == "HHHH":
    ID["pol_pair_ID"] = 0
  elif Pol_pair == "HVHV":
    ID["pol_pair_ID"] = 1
  elif Pol_pair == "VHVH":
    ID["pol_pair_ID"] = 2
  elif Pol_pair == "VVVV":
    ID["pol_pair_ID"] = 3
  elif Pol_pair == "HHVV":
    ID["pol_pair_ID"] = 4
  else:
    raise ValueError("Polarization pair " + Pol_pair + " does not exist in SARSIM. Available choices:{HHHH,HVHV,VHVH,VVVV,HHVV}")
  
  if Band == "P":
    ID["band_ID"] = 0
  elif Band == "L":
    ID["band_ID"] = 1
  else:
    raise ValueError(Band + "-Band does not exist in SARSIM. Available choices:{P,L}")
  
  if Sensor == "Airborne":
    ID["sensor_ID"] = 0
  elif Sensor == "Spaceborne":
    ID["sensor_ID"] = 1
  else:
    raise ValueError("Sensor " + Sensor + " does not exist in SARSIM. Available choices:{Airborne,Spaceborne}")

  return ID
####################################################################################################

def SARSIM_PLUS_param(ID):
  param = {}
  
  if ID["operator_ID"] == 0: # DLR
    if ID["site_ID"] == 0:
      if ID["band_ID"] == 0: # DLR Lope P
        param["x_ax_min"] = 0
        param["x_ax_max"] = 10933.8946
        param["dx"] = 6.3643
        param["Nx"] = 1719
        param["y_ax_min"] = 2419.5311
        param["y_ax_max"] = 9307.1404
        param["dy"] = 5.9944
        param["Ny"] = 1150
        param["Multilook"] = "25mx25m"
        param["z_axis"] = np.arange(-10,51)
      else: # DLR LOPE L
        param["x_ax_min"] = 15.0069
        param["x_ax_max"] = 1.0920e+04
        param["dx"] = 5
        param["Nx"] = 2182
        param["y_ax_min"] = 2.3475e+03
        param["y_ax_max"] = 9.2375e+03
        param["dy"] = 5
        param["Ny"] = 1379
        param["Multilook"] = "25mx25m"
        param["z_axis"] = np.arange(-10,71)
        if ID["sensor_ID"] == 1: # spaceborne
          param["Multilook"] = "50mx50m"
    elif ID["site_ID"] == 1: # DLR Remningstorp P
      param["x_ax_min"] = 0
      param["x_ax_max"] = 5887.44
      param["dx"] = 6.66
      param["Nx"] = 885
      param["y_ax_min"] = 1791.6718
      param["y_ax_max"] = 5190.3605
      param["dy"] = 5.9942
      param["Ny"] = 568
      param["Multilook"] = "25mx25m"
      param["z_axis"] = np.arange(-10,51)
    elif ID["site_ID"] == 2: # DLR Traunstein L
      param["x_ax_min"] = 15.0089
      param["x_ax_max"] = 8.3700e+03
      param["dx"] = 5
      param["Nx"] = 1672
      param["y_ax_min"] = 2.2739e+03
      param["y_ax_max"] = 6.8539e+03
      param["dy"] = 5
      param["Ny"] = 917
      param["Multilook"] = "25mx25m"
      param["z_axis"] = np.arange(-10,51)
      if ID["sensor_ID"] == 1: # spaceborne
        param["Multilook"] = "50mx50m"
    elif ID["site_ID"] == 3: # DLR Krycklan SW
      param["x_ax_min"] = -4.235599448975323e+03
      param["x_ax_max"] = 4.641272129303370e+03
      param["dx"] = 5.169989270983933
      param["Nx"] = 1718
      param["y_ax_min"] = 1.594308467008164e+03
      param["y_ax_max"] = 5.359308467008164e+03
      param["dy"] = 5
      param["Ny"] = 754
      param["Multilook"] = "25mx25m"
      param["z_axis"] = np.arange(-10,51)
      if ID["sensor_ID"] == 1: # spaceborne
        param["Multilook"] = "50mx50m"
    elif ID["site_ID"] == 4: # DLR Krycklan NE
      param["x_ax_min"] = -3.514834224505171e+03
      param["x_ax_max"] = 5.545322153643585e+03
      param["dx"] = 5.225003678286157
      param["Nx"] = 1735
      param["y_ax_min"] = 1.693522152551585e+03
      param["y_ax_max"] = 5.338522152551584e+03
      param["dy"] = 5
      param["Ny"] = 730
      param["Multilook"] = "25mx25m"
      param["z_axis"] = np.arange(-10,51)
      if ID["sensor_ID"] == 1:
        param["Multilook"] = "50mx50m"
  elif ID["operator_ID"] == 1:
    if ID["band_ID"] == 0: # ONERA Lope P
      param["x_ax_min"] = 0
      param["x_ax_max"] = 10368
      param["dx"] = 6
      param["Nx"] = 1729
      param["y_ax_min"] = 2388.4467
      param["y_ax_max"] = 7560.2036
      param["dy"] = 7.193
      param["Ny"] = 720
      param["Multilook"] = "25mx25m"
      param["z_axis"] = np.arange(-10,51)
    else:     # ONERA Lope L
      param["x_ax_min"] = 0
      param["x_ax_max"] = 11500
      param["dx"] = 4
      param["Nx"] = 2876
      param["y_ax_min"] = 3031.5165
      param["y_ax_max"] = 7351.5156
      param["dy"] = 4
      param["Ny"] = 1081
      param["Multilook"] = "25mx25m"
      param["z_axis"] = np.arange(-10,51)
      if ID["sensor_ID"] == 1: # spaceborne
        param["Multilook"] = "50mx50m"
        param["z_axis"] = np.arange(-15,61)
  elif ID["operator_ID"] == 2: # JPL Lope L
    param["x_ax_min"] = 0
    param["x_ax_max"] = 5999.4
    param["dx"] = 6.6
    param["Nx"] = 910
    param["y_ax_min"] = 4875
    param["y_ax_max"] = 13443
    param["dy"] = 7
    param["Ny"] = 1225
    param["Multilook"] = "25mx25m"
    param["z_axis"] = np.arange(-10,51)
    if ID["sensor_ID"] == 1:
      param["Multilook"] = "50mx50m"
      param["z_axis"] = np.arange(-15,61)
  
  param["x_axis"] = np.linspace(param["x_ax_min"], param["x_ax_max"], param["Nx"])
  param["y_axis"] = np.linspace(param["y_ax_min"], param["y_ax_max"], param["Ny"])
  param["Nz"] = len(param["z_axis"])
  
  return param
###################################################################################################

def SARSIM_getstring(folder = None, operator_ID = None, band_ID = None, site_ID = None, pol_ID = None, pol_pair_ID = None, part_ind = None, type_ID = None):
  
  # generate strings to build paths 
  type_str_vect = ["SLC", "Kz", "DTM_SAR", "DTM_ground", "Tomo_Intensity", "Tomo_Coherence"]
  site_str_vect = ["Lope", "Remningstorp", "Traunstein", "Krycklan_SW", "Krycklan_NE"]
  operator_str_vect = ["DLR", "ONERA", "JPL"]
  band_str_vect = ["P", "L"]
  pol_str_vect = ["HH", "HV", "VH", "VV"]
  pol_pair_str_vect = ["HHHH","HVHV", "VHVH", "VVVV", "HHVV"]
  part_str_vect = ["Real", "Imag"]
  folders_str_vect = ["SLC_airborne\\", "Kz\\","DTM\\", "Tomo_airborne\\", "SLC_spaceborne\\", "Tomo_spaceborne\\"]

  if folder != None:
    string = folders_str_vect[folder]
  elif operator_ID != None:
    string = operator_str_vect[operator_ID]
  elif band_ID != None:
    string = band_str_vect[band_ID]
  elif site_ID != None:
    string = site_str_vect[site_ID]
  elif pol_ID != None:
    string = pol_str_vect[pol_ID]
  elif pol_pair_ID != None:
    string = pol_pair_str_vect[pol_pair_ID]
  elif part_ind != None:
    string = part_str_vect[part_ind]
  elif type_ID != None:
    string = type_str_vect[type_ID]

  return string

####################################################################################################

def LOAD_SINGLE_TOMOSAR_HOR_SEC(ID,param):
  k = (np.abs(ID["Z"]-param["z_axis"])).argmin()
  Section_height = param["z_axis"][k]
  ID["Section_height"] = Section_height
  
  local_folder_str = SARSIM_getstring(folder = ((ID["sensor_ID"] == 0)*3 + (ID["sensor_ID"] == 1)*5))
  curr_folder = ID["parent_folder"] + local_folder_str + SARSIM_getstring(pol_pair_ID= ID["pol_pair_ID"]) + "\\"
  curr_type_ID = (ID["pol_pair_ID"] < 4)*4 + (ID["pol_pair_ID"] == 4)*5
  #sensor_str = ["Airborne", "Spaceborne"]
  
  
  if ID["pol_pair_ID"] < 4:
    slice_filename = curr_folder + SARSIM_getstring(type_ID=curr_type_ID) + "_" + SARSIM_getstring(site_ID=ID["site_ID"])\
    + "_" + SARSIM_getstring(operator_ID = ID["operator_ID"]) + "_" + SARSIM_getstring(band_ID = ID["band_ID"]) + "_Band_"\
    + SARSIM_getstring(pol_pair_ID= ID["pol_pair_ID"]) + "_z=" + np.str(Section_height) + ".tiff"
    print("Loading " + slice_filename + "...")
    dataset = gdal.Open(slice_filename, gdal.GA_ReadOnly)
    band = dataset.GetRasterBand(1)
    tomo_slice = band.ReadAsArray()
    dataset = None
    print("Loading done")
  else:
    tomo_slice = np.zeros((param["Nx"], param["Ny"]))
    for part_ind in np.arange(0,2):
      slice_filename = curr_folder + SARSIM_getstring(type_ID=curr_type_ID) + "_" + SARSIM_getstring(site_ID=ID["site_ID"])\
      + "_" + SARSIM_getstring(operator_ID = ID["operator_ID"]) + "_" + SARSIM_getstring(band_ID = ID["band_ID"]) + "_Band_"\
      + SARSIM_getstring(pol_pair_ID= ID["pol_pair_ID"]) + "_z=" + np.str(Section_height) + "_"\
      + SARSIM_getstring(part_ind =part_ind) + "_part.tiff"
      
      print("Loading " + slice_filename + "...") 
      dataset = gdal.Open(slice_filename, gdal.GA_ReadOnly)
      band = dataset.GetRasterBand(1)
      tomo_slice_temp = band.ReadAsArray()
      dataset = None
      tomo_slice = tomo_slice +tomo_slice_temp*np.exp(1j*np.pi/2*(part_ind==1))
      print("Loading done")
  
  # L-Band DLR airborne tomo-cubes in Lope and Traunstein need to be rotated by
  # 90 degree to be consistent with the other datasets
  if ID["band_ID"] == 1 and ID["operator_ID"] == 0  and (ID["site_ID"] == 0\
       or ID["site_ID"] == 2) and ID["sensor_ID"] == 0:
    tomo_slice = np.rot90(np.flipud(tomo_slice),-1)
    
  return tomo_slice

####################################################################################################

def LOAD_GROUND_DTM(ID):
  
  dtm_folder = ID["parent_folder"] + "DTM\\"
  dtm_filename = "DTM_ground_" + ID["dataset"]["Site"] + "_" + ID["dataset"]["Operator"]\
  + "_" + ID["dataset"]["Band"] + "_Band.tiff"
  g2s_filename = "Ground2SAR_" + ID["dataset"]["Site"] + "_" + ID["dataset"]["Operator"]\
  + "_" + ID["dataset"]["Band"] + "_Band.tiff"
  dtm_path = dtm_folder+dtm_filename
  g2s_path = dtm_folder+g2s_filename
  
  dtmset = gdal.Open(dtm_path, gdal.GA_ReadOnly)
  band1 = dtmset.GetRasterBand(1)
  dtm_ground_coord = band1.ReadAsArray()
  dtmset = None
  
  g2sset = gdal.Open(g2s_path, gdal.GA_ReadOnly)
  band2 = g2sset.GetRasterBand(1)
  ground_2_sland_range_map = band2.ReadAsArray()
  g2sset = None
  
  return dtm_ground_coord, ground_2_sland_range_map

####################################################################################################
  
def LOAD_ALL_TOMOSAR_HOR_SEC(ID, param):
  TOMOSAR_VOX = np.zeros((param["Ny"],param["Nx"], param["Nz"]))
  for n in np.arange(0,param["Nz"]):
    ID["Z"] = param["z_axis"][n]
    tomo_slice = LOAD_SINGLE_TOMOSAR_HOR_SEC(ID, param)
    TOMOSAR_VOX[:,:,n] = tomo_slice

  return TOMOSAR_VOX

####################################################################################################


# 2) Processing

## 2 A) Load and display a single horizontal section

### Data Selection and loading

define the dataset you want to analyse by passing the arguments in the following cell:<br>
<br>
ID = SARSIM_PLUS_DATA_SELECTION(sarsim_path,'Operator','Site','Pol_pair','Band','Sensor', Section)<br><br>
Operator = {DLR,ONERA,JPL}<br>
Site = {Lope, Remningstorp, Traunstein, Krycklan_SW, Krycklan_NE}<br>
Pol_pair = {HHHH,HVHV,VHVH,VVVV,HHVV}<br>
Band = {P,L}<br>
Sensor = {Airborne,Spaceborne}<br>
Section: Height of the horizontal section (in meters) w.r.t. the DTM<br><br>
Not every combination is available.

In [ ]:
# Data selection
ID = SARSIM_PLUS_DATA_SELECTION(sarsim_path,"DLR","Lope","VVVV", "P", "Airborne",30)

In [ ]:
# Load SAR and ancillary data
param = SARSIM_PLUS_param(ID)
tomo_slice = LOAD_SINGLE_TOMOSAR_HOR_SEC(ID,param)
dtm_ground_coord,ground_2_slant_range_map = LOAD_GROUND_DTM(ID)

### Display loaded data

In [ ]:
data_string = ID["dataset"]["Operator"] + " " + ID["dataset"]["Site"] + " "\
+ ID["dataset"]["Pol_pair"] + " " + ID["dataset"]["Band"] + " "\
+ ID["dataset"]["Sensor"]


extent = (np.min(param["x_axis"]),np.max(param["x_axis"]), np.max(param["y_axis"]), np.min(param["y_axis"]))
if ID["dataset"]["Pol_pair"] == "HHVV":
  
  fig = plt.figure(1)
  ax1 = fig.add_subplot(111)
  map1 = ax1.imshow(abs(tomo_slice), extent = extent, aspect = "equal", cmap = "viridis", vmin = 0, vmax = 1)
  plt.colorbar(map1)
  ax1.set_title("HHVV coherence at " + np.str(ID["Section_height"]) + " m above the ground. \n" + data_string)
  ax1.set_xlabel("Azimuth [m]")
  ax1.set_ylabel("Ground range [m]")
  plt.tight_layout()
  
  fig = plt.figure(2)
  ax2 = fig.add_subplot(111)
  map2 = ax2.imshow(np.angle(tomo_slice), extent = extent, aspect = "equal", cmap = "viridis", vmin = -np.pi, vmax = np.pi)
  plt.colorbar(map2)
  ax2.set_title("HHVV phase at " + np.str(ID["Section_height"]) + " m above the ground. \n" + data_string)
  ax2.set_xlabel("Azimuth [m]")
  ax2.set_ylabel("Ground range [m]")
  plt.tight_layout()
  
  
else:
  tomo_slice[tomo_slice<0] = 0
  fig = plt.figure(3)
  ax3 = fig.add_subplot(111)
  max_ts = np.nanmax(10*np.log10(abs(tomo_slice)))
  map3 = ax3.imshow(10*np.log10(abs(tomo_slice)), extent = extent, aspect = "equal", cmap = "viridis", vmin = -40+max_ts, vmax = 0+max_ts)
  plt.colorbar(map3)
  ax3.set_title("Tomographic intensity at " + np.str(ID["Section_height"]) + " m above the ground. \n" + data_string)
  ax3.set_xlabel("Azimuth [m]")
  ax3.set_ylabel("Ground range [m]")
  plt.tight_layout()

## 2 B) Load all horizontal sections and display vertical sections

### Data selection and loading

In [ ]:
# Data selection
ID = SARSIM_PLUS_DATA_SELECTION(sarsim_path,"DLR","Lope","VVVV", "P", "Airborne","All")

In [ ]:
# Load SAR and ancillary data
param = SARSIM_PLUS_param(ID)
TOMOSAR_VOX = LOAD_ALL_TOMOSAR_HOR_SEC(ID,param)

### Display vertical sections at fixed azimuth position

In [ ]:
# Extract vertical section
x0 = 1500 # azimuth position
tomo_slice = TOMOSAR_VOX[:,x0,:].T

In [ ]:
data_string = ID["dataset"]["Operator"] + " " + ID["dataset"]["Site"] + " "\
+ ID["dataset"]["Pol_pair"] + " " + ID["dataset"]["Band"] + " "\
+ ID["dataset"]["Sensor"]

Display section position over intensity

In [ ]:
Avg_intensity = np.nanmean(TOMOSAR_VOX,2)
Avg_intensity[Avg_intensity<0] = 0

# optimize contrast
Avg_intensity_con = Avg_intensity[~np.isnan(Avg_intensity)]
Avg_intensity_con = 10*np.log10(Avg_intensity_con)
Avg_intensity_con = Avg_intensity_con[~np.isinf(Avg_intensity_con)]
vmin = np.percentile(Avg_intensity_con,2)
vmax = np.percentile(Avg_intensity_con,98)

extent = ((np.min(param["x_axis"]), np.max(param["x_axis"]), np.max(param["y_axis"]), np.min(param["y_axis"])))
fig = plt.figure(4)
ax4 = fig.add_subplot(111)
map4 = ax4.imshow(10*np.log10(Avg_intensity), extent = extent, aspect = "equal", cmap = "viridis", vmin = vmin, vmax = vmax)
ax4.plot([param["x_axis"][x0],param["x_axis"][x0]], [param["y_axis"][0],param["y_axis"][-1]], "k-")
plt.colorbar(map4)
ax4.set_title("Average Intensity [dB] " + data_string)
ax4.set_xlabel("Azimuth [m]")
ax4.set_ylabel("range [m]")
plt.tight_layout()

Display vertical section

In [ ]:
norm_function = np.expand_dims(np.sum(tomo_slice,0),0)
tomo_slice_norm = tomo_slice/(np.ones((param["Nz"],1))@norm_function)

# set the colorscale to optimize contrast
vmax = np.percentile(tomo_slice_norm[~np.isnan(tomo_slice_norm)],98)

extent = (np.min(param["y_axis"]),np.max(param["y_axis"]),np.max(param["z_axis"]),np.min(param["z_axis"]))
fig = plt.figure(5)
ax5 = fig.add_subplot(211)
map5 = plt.imshow(tomo_slice_norm,extent = extent,cmap = "viridis", aspect = "auto", vmin = 0, vmax = vmax)
plt.gca().invert_yaxis()
plt.colorbar(map5)
ax5.set_title("Vertical Section " + data_string)
ax5.set_xlabel("Ground range [m]")
ax5.set_ylabel("Height [m]")
plt.tight_layout()